In [ ]:
import numpy as np
import scipy
import scipy.linalg as la
import functools as ft
import matplotlib.pyplot as plt
import tinyarray as ta
import kwant
import matplotlib.pyplot as plt
import pickle
from inspect import getsource
print(kwant.__version__)


from lcao import L_matrices, lcao_term
from hamiltonians import sigmas

# Calculations for 18 orbital SnPbTe

In [ ]:
import hpc05

In [ ]:
hpc05.kill_remote_ipcluster()

In [ ]:
client, dview, lview = hpc05.start_remote_and_connect(26, profile='pbs_32GB', timeout=600,
                                                      env_path='/home/dvarjas/.conda/envs/kwant_dev',
                                                      folder='~/disorder_invariants/code/',
                                                      # kill_old_ipcluster=False,
                                                     )

In [ ]:
%%px --local
import itertools as it
import scipy
import scipy.linalg as la
import numpy as np
import copy
import functools as ft
import pickle

# from scipy.integrate import cumtrapz
# from scipy.interpolate import interp1d
# from scipy.optimize import brentq

import kwant
print(kwant.__version__)
pickle.dump(kwant.__version__, open('version.pickle', 'wb'))
# assert kwant.__version__ == '1.4.0a1.dev57+g402a0ca'
from hamiltonians import SnTe_18band_disorder, site_type, SnPbTe_params
from mirror_chern import mirror_chern, make_window, random_phase_vecs, pg_op, M_cubic, UM_spd
from kpm_funcs import position_operator

In [ ]:
%%px --local
# Make a slab with PBC
# surface normal
n = np.array([1, 1, 0])
n11 = np.array([1, -1, 0])
nz = np.array([0, 0, 1])
# thickness (number of atomic layers - 1)
W = 40
L11 = 80
Lz = 120

num_vectors = 5
num_moments = 5000

salts = range(6, 12)
num_x = 26
x_array = np.linspace(0, 1, num_x)

# Set Fermi level based on half filling. Integration is inaccurate unless 
# many moments are used in the spectrum, which is expensive.
# def set_fermi_level(spectrum, filling):
#     energies = spectrum.energies
#     cum_DoS = cumtrapz(spectrum(energies), energies, initial=0)
#     total = spectrum.integrate()
#     f = interp1d(energies, cum_DoS - total * filling, kind='quadratic')
#     return brentq(f, min(energies), max(energies))

def set_fermi_level(doping):
    # Set Fermi level manually, see DoS plot.
    return 0.135 + 0.09 * doping - 0.13 * doping**2


def make_operators(doping, salt='salt'):
    syst = SnTe_18band_disorder()

    # Build film using syst
    film = kwant.Builder(kwant.lattice.TranslationalSymmetry(W * n, L11 * n11, Lz * nz))

    film.fill(syst, lambda site: True, start=np.zeros(3));
    filmw = kwant.wraparound.wraparound(film)   
    filmw = filmw.finalized()

    M_trf = ft.partial(M_cubic, n=n)
    UM = UM_spd(n)
    M = pg_op(filmw, M_trf, UM)

    pars = SnPbTe_params.copy()
    # to_fd = filmw._wrapped_symmetry.to_fd

    # window half the size
    win_L11 = L11//2
    win_Lz = Lz//2
    A = win_L11 * win_Lz * np.sqrt(2)

    # Make it periodic with the size of the window
    to_fd = kwant.lattice.TranslationalSymmetry(W * n, win_L11 * n11, win_Lz * nz).to_fd

    pars['site_type'] = ft.partial(site_type, doping=doping, n=n, to_fd=to_fd, salt=salt)
    pars['k_x'] = pars['k_y'] = pars['k_z'] = 0

    H = filmw.hamiltonian_submatrix(params=pars, sparse=True)
    ham_size = H.shape[0]
    norbs = filmw.sites[0].family.norbs

    x0, y0, z0 = position_operator(filmw)
    x = 1/np.sqrt(2) * (x0 - y0)
    y = z0

    def shape1(site):
        tag = site.tag
        tag11 = np.dot(tag, n11) // np.dot(n11, n11)
        tagz = np.dot(tag, nz) // np.dot(nz, nz)
        tagn = (np.dot(tag, n) % (W * n.dot(n))) // np.dot(n, n)
        return (-win_L11/2 + L11/2 < tag11 <= win_L11/2 + L11/2 and
                -win_Lz/2 + Lz/2 < tagz <= win_Lz/2 + Lz/2 and
                tagn < W//2)
    window1 = make_window(filmw, shape1)
    
    def shape2(site):
        tag = site.tag
        tag11 = np.dot(tag, n11) // np.dot(n11, n11)
        tagz = np.dot(tag, nz) // np.dot(nz, nz)
        tagn = (np.dot(tag, n) % (W * n.dot(n))) // np.dot(n, n)
        return (-win_L11/2 + L11/2 < tag11 <= win_L11/2 + L11/2 and
                -win_Lz/2 + Lz/2 < tagz <= win_Lz/2 + Lz/2 and
                tagn >= W//2)
    window2 = make_window(filmw, shape2)

    return H, M, x, y, [window1, window2], pars, A
    

def job(doping, salt='salt'):
    salt = str(salt)
    print(doping)
    
    H, M, x, y, windows, pars, A = make_operators(doping, salt)
    
    # spectrum = kwant.kpm.SpectralDensity(H, num_moments=num_moments, params=pars)
    # filling = 5/18
    # e_F = set_fermi_level(spectrum, filling)
    e_F = set_fermi_level(doping)
    # filling = spectrum.integrate(lambda x: x < e_F) / spectrum.integrate()
    # mindos = spectrum(e_F)

    C_list = [mirror_chern(H, x, y, Mz=M, vectors=num_vectors,
                          e_F=e_F, kpm_params=dict(num_moments=num_moments),
                          params=pars, bounds=None, window=window, return_std=False)
              for window in windows]

    C_list = np.array(C_list)
    Cs = np.sum(C_list, axis=0) / A
    C = np.mean(Cs)
    C_std = np.std(Cs)
    pickle.dump(dict(params=SnPbTe_params, 
                 doping=doping,
                 e_F=e_F,
                 W=W,
                 L11=L11,
                 Lz=Lz,
                 salt=salt,
                 Cs=Cs,
                 num_vectors=num_vectors,
                 num_moments=num_moments,),
            open('disorder_invariants/data/SnTe18_MC_'+str(W)+'x'+str(L11)+'x'+str(Lz)
                 +'/Mirror_Chern_doped_SnTe_18band_eff_'+str(salt)+'_'+str(doping)+'.pickle', 'wb'))
    print(C, C_std)
    return C, C_std, # filling, e_F, mindos

In [ ]:
# omit combinations that were already used
salts = range(0, 7)
_, existing = get_data('data/SnTe18_MC_'+str(W)+'x'+str(L11)+'x'+str(Lz)+'.pickle')
xss = [(x, s) for s, x in it.product(salts, x_array) if not str(s) in existing[x]]
len(xss)

In [ ]:
result = lview.map_async(job, *zip(*xss))

In [ ]:
result.wait_interactive()

In [ ]:
all([err is None for err in result.error])

### Organize results

In [ ]:
directories = ! ls -d ../data/SnTe18_MC_*/

for directory in directories:
    # ! rsync -rP hpc05:disorder_invariants/code/{directory} data/
    size = directory.split('SnTe18_MC_')[-1]
    size = size.split('/')[0]
    data = defaultdict(list)
    try:
        old_data = pickle.load(open(directory[:-1] + '.pickle', 'rb'))
    except:
        pass
    else:
        data.update(old_data)
    num_moments = None
    files = ! ls {directory}Mirror_Chern_doped_SnTe_18band_eff*
    for file in files:
        dat = pickle.load(open(file, 'rb'))
        W, L11, Lz = size.split('x')
        doping = dat['doping']
        salt = dat['salt']
        if not (dat['L11'] == int(L11) and dat['L11'] == int(L11) and dat['Lz'] == int(Lz) and
                (num_moments is None or dat['num_moments'] == num_moments) and
                np.isclose(dat['e_F'], set_fermi_level(doping))):
            assert False
        Cs = list(dat['Cs'])
        # print(directory, (doping, salt) )
        if (not (doping, salt) in data) or (not Cs[0] in data[(doping, salt)]):
            data[(doping, salt)] += Cs
        num_moments = dat['num_moments']
    data['num_moments'] = num_moments
    data['set_fermi_level'] = getsource(set_fermi_level)
    data['W'] = W
    data['L11'] = L11
    data['Lz'] = Lz
    pickle.dump(data, open(directory[:-1] + '.pickle', 'wb'))

## DoS

In [ ]:
%%px --local
# Make a slab with PBC
# surface normal
n = np.array([1, 1, 0])
n11 = np.array([1, -1, 0])
nz = np.array([0, 0, 1])

# thickness (number of atomic layers - 1)
W = 40
L11 = 40
Lz = 60

num_vectors = 5
num_moments = 5000

num_x = 26
x_array = np.linspace(0, 1, num_x)

def job2(doping):
    print(doping)
    
    syst = SnTe_18band_disorder()

    # Build film using syst
    film = kwant.Builder(kwant.lattice.TranslationalSymmetry(W * n, L11 * n11, Lz * nz))

    film.fill(syst, lambda site: True, start=np.zeros(3));
    filmw = kwant.wraparound.wraparound(film)   
    filmw = filmw.finalized()

    pars = SnPbTe_params.copy()
    to_fd = filmw._wrapped_symmetry.to_fd

    pars['site_type'] = ft.partial(site_type, doping=doping, n=n, to_fd=to_fd, salt='salt')
    pars['k_x'] = pars['k_y'] = pars['k_z'] = 0
    
    spectrum = kwant.kpm.SpectralDensity(filmw, num_moments=num_moments, params=pars)

    return spectrum()

In [ ]:
result = lview.map_async(job2, x_array)

In [ ]:
result.wait_interactive()

In [ ]:
all([err is None for err in result.error])

In [ ]:
spec_vs_x = np.array(result.get())

In [ ]:
pickle.dump(dict(params=SnPbTe_params,
                 x_array=x_array,
                 W=W, L11=L11, Lz=Lz,
                 num_moments=num_moments,
                 disorder_realizations=1,
                 spec_vs_x=spec_vs_x,
                 ),
           open('../data/DoS_SnPbTe18.pickle', 'wb'))

In [ ]:
plt.figure(figsize=(5, 5))
for i, (es, doss) in enumerate(spec_vs_x):
    plt.plot(es, doss.real + 10000*x_array[i], c='k')
    # plt.plot([0.13 + 0.06 * x_array[i] - 0.08 * x_array[i]**2], [10000*x_array[i]], '.', c='r')
plt.plot(0.135 + 0.09 * x_array - 0.13 * x_array**2, 10000*x_array, '-', c='r')
# plt.plot(0.05 + 0.1 * x_array - 0.0 * x_array**2, 10000*x_array, '-', c='b')
# plt.plot(0.22 + 0.08 * x_array - 0.26 * x_array**2, 10000*x_array, '-', c='b')
plt.xlim(-0.1, 0.4)
plt.ylim(-100, 12000)